# Base Functions

In [1]:
import json
import re
import socket
from mac_vendor_lookup import MacLookup
# from mac_vendor_lookup import AsyncMacLookup

def update_ouis(nm_path):
    '''
    Creates a dataframe of vendor OUIs. List is taken from
    https://standards-oui.ieee.org/. It is downloaded ahead of time to save
    time.
    
    There is a Python library for it, but it is prohibitively slow.
    
    To update the OUIs, save the data in https://standards-oui.ieee.org/ to
    Net-Manage/ouis.txt.
    
    Note: It might seem inefficient to parse the OUIs and create the
    dataframe as needed. However, it only takes 250ms~ to do, and the DataFrame
    size is only 500KB~. Therefore, I find the overhead acceptable.

    Args:
        nm_path (str): Path to the Net-Manage repository.

    Returns:
        df_ouis (DataFrame): A DataFrame containing the vendor OUIs
    '''
    with open(f'{nm_path}/ouis.txt', 'r') as f:
        data = f.read()
    pattern = '.*base 16.*'
    data = re.findall(pattern, data)
    data = [[l.split()[0], l.split('\t')[-1]] for l in data]
    df_ouis = pd.DataFrame(data=data, columns=['base', 'vendor'])
    return df_ouis
    

# def find_mac_vendor(mac_address):
#     return MacLookup().lookup(mac_address)

def find_mac_vendor(addresses, df_ouis):
    '''
    Finds the vendor OUI for a list of MAC addresses.
    
    Note: This function could be improved by multi-threading. However, in
    testing I found the performance in its current state to be acceptable. It
    took about 30 seconds to get the vendors for 16,697 MAC addresses. That
    DOES include the time it took for Ansible to connect to the devices and get
    the CAM tables.
    
    Args:
        addresses (list): A list of MAC adddresses.

    Returns:
        vendors (list):   A list of vendors, ordered the same as 'addresses'
    '''
    # Convert MAC addresses to base 16
    addresses = [m.replace(':', str()) for m in addresses]
    addresses = [m.replace('-', str()) for m in addresses]
    addresses = [m.replace('.', str()) for m in addresses]
    addresses = [m[:6].upper() for m in addresses]
    # Create a list to store vendors
    vendors = list()
    
    # Create a list to store unknown vendors. This is used to keep from
    # repeatedly searching for vendor OUIs that are unknown
    unknown = list()
    
    # Create a dict to store known vendors. This is used to keep from
    # repeatedly searching df_ouis for vendors that are already discovered
    known = dict()
    
    # Search df_ouis for the vendor, add it to 'vendors', and return it
    for mac in addresses:
        vendor = known.get(mac)
        if not vendor:
            if mac in unknown:
                vendor = 'unknown'
            else:
                vendor_row = df_ouis.loc[df_ouis['base'] == mac]
                if len(vendor_row) > 0:
                    vendor = vendor_row['vendor'].values[0]
                else:
                    vendor = 'unknown'
                    unknown.append(mac)
        vendors.append(vendor)
    return vendors
    

def run_test(ansible_os,
             test_name,
             username,
             password,
             hostgroup,
             play_path,
             private_data_dir,
             nm_path,
             ansible_timeout='300'):
    '''
    This function calls the test that the user requested.

    Args:
        os (str):               The ansible_network_os variable
        test_name (str):        The name of the test that the user requested
        username (str):         The username to login to devices
        password (str):         The password to login to devices
        host_group (str):       The inventory host group
        play_path (str):        The path to the playbooks directory
        private_data_dir (str): The path to the Ansible private data directory
        nm_path (str):          The path to the Net-Manage repository
        interface (str):        The interface (defaults to all interfaces)
    '''
    if test_name == 'cam_table':
        if ansible_os == 'cisco.ios.ios':
            result = get_ios_cam_table(username,
                                       password,
                                       hostgroup,
                                       play_path,
                                       private_data_dir,
                                       nm_path)
        if ansible_os == 'cisco.nxos.nxos':
            result = get_nxos_cam_table(username,
                                        password,
                                        hostgroup,
                                        play_path,
                                        private_data_dir,
                                        nm_path)
            
    if test_name == 'arp_table':
        if ansible_os == 'paloaltonetworks.panos':
            result = get_panos_arp_table(username,
                                         password,
                                         hostgroup,
                                         play_path,
                                         private_data_dir)

    if test_name == 'interface_description':
        if ansible_os == 'cisco.ios.ios':
            result = get_ios_inf_descs(username,
                                       password,
                                       hostgroup,
                                       play_path,
                                       private_data_dir)
            
    if test_name == 'find_uplink_by_ip':
        if ansible_os == 'cisco.ios.ios':
            result = cisco_ios_find_uplink_by_ip(username,
                                                 password,
                                                 hostgroup,
                                                 play_path,
                                                 private_data_dir)

    if test_name == 'pre_post_checks':
        if ansible_os == 'cisco.nxos.nxos':
            result = cisco_nxos_run_checks(username,
                                           password,
                                           hostgroup,
                                           play_path,
                                           private_data_dir)
    return result

# Palo Alto Tests

## arp_table

In [ ]:
def get_panos_arp_table(username,
                        password,
                        host_group,
                        play_path,
                        private_data_dir,
                        interface=None,
                        reverse_dns=False):
    '''
    Gets the ARP table from a Palo Alto firewall.

    Args:
        username (str):         The username to login to devices
        password (str):         The password to login to devices
        host_group (str):       The inventory host group
        play_path (str):        The path to the playbooks directory
        private_data_dir (str): The path to the Ansible private data directory
        interface (str):        The interface (defaults to all interfaces)
        reverse_dns (bool):     Whether to run a reverse DNS lookup. Defaults
                                to False because the test can take several
                                minutes on large ARP tables.

    Returns:
        df_arp (DataFrame):     The ARP table
    '''
    if interface:
        cmd = f'show arp {interface}'
    else:
        cmd = 'show arp all'
    extravars = {'username': username,
                 'password': password,
                 'host_group': host_group,
                 'command': cmd}
    
    playbook = f'{play_path}/palo_alto_get_arp_table.yml'
    runner = ansible_runner.run(private_data_dir=private_data_dir,
                                playbook=playbook,
                                extravars=extravars)
    
    for event in runner.events:
        if event['event'] == 'runner_on_ok':
            event_data = event['event_data']
            device = event_data['remote_addr']
            output = event_data['res']['stdout']
            
            output = json.loads(output)
            output = output['response']['result']['entries']['entry']
            
            arp_table = list()
            for item in output:
                address = item['ip']
                age = item['ttl']
                mac = item['mac']
                inf = item['interface']
                # Lookup the OUI vendor
                try:
                    vendor = find_mac_vendor(mac)
                except Exception:
                    vendor = 'unknown'
                row = [device, address, age, mac, inf, vendor]
                # Perform a reverse DNS lookup if requested
                if reverse_dns:
                    try:
                        rdns = socket.getnameinfo((address, 0), 0)[0]
                    except Exception:
                        rdns = 'unknown'
                    row.append(rdns)
                arp_table.append(row)
                
    cols = ['Device',
            'Address',
            'Age',
            'MAC Address',
            'Interface',
            'Vendor']
    if reverse_dns:
        cols.append('Reverse DNS')

    df_arp = pd.DataFrame(data=arp_table, columns=cols)            
    return df_arp

# IOS Tests

## cdp_neighbors

In [1]:
def get_ios_cdp_neighbors(username,
                          password,
                          host_group,
                          play_path,
                          private_data_dir,
                          interface=str()):
    '''
    Gets the CDP neighbors for a Cisco IOS device.

    Args:
        username (str):         The username to login to devices
        password (str):         The password to login to devices
        host_group (str):       The inventory host group
        play_path (str):        The path to the playbooks directory
        private_data_dir (str): The path to the Ansible private data directory
        interface (str):        The interface to get the neighbor entry for. If
                                not specified, it will get all neighbors.

    Returns:
        df_cdp (DataFrame):     A DataFrame containing the CDP neighbors
    '''
    cmd = 'show cdp neighbor detail | include Device ID|Interface'
    extravars = {'username': username,
                 'password': password,
                 'host_group': host_group,
                 'commands': cmd}

    # Execute the command
    playbook = f'{play_path}/cisco_ios_run_commands.yml'
    runner = ansible_runner.run(private_data_dir=private_data_dir,
                                playbook=playbook,
                                extravars=extravars)

    # Parse the results
    cdp_data = list()
    for event in runner.events:
        if event['event'] == 'runner_on_ok':
            event_data = event['event_data']
            
            device = event_data['remote_addr']
            
            output = event_data['res']['stdout'][0].split('\n')
            pos = 1  # Used to account for multiple connections to same device,
                     # because in that case index to get next line won't work
            for line in output:
                if 'Device ID' in line:
                    remote_device = line.split('(')[0].split()[2].split('.')[0]
                    local_inf = output[pos].split()[1].strip(',')
                    remote_inf = output[pos].split()[-1]
                    row = [device, local_inf, remote_device, remote_inf]
                    cdp_data.append(row)
                pos += 1
    # Create a dataframe from cdp_data and return the results
    cols = ['Device', 'Local Inf', 'Neighbor', 'Remote Inf']
    df_cdp = pd.DataFrame(data=cdp_data, columns=cols)
    return df_cdp    

## cam_table

In [ ]:
def get_ios_cam_table(username,
                      password,
                      host_group,
                      play_path,
                      private_data_dir,
                      nm_path,
                      interface=None):
    '''
    Gets the IOS CAM table and adds the vendor OUI.

    Args:
        username (str):         The username to login to devices
        password (str):         The password to login to devices
        host_group (str):       The inventory host group
        play_path (str):        The path to the playbooks directory
        private_data_dir (str): The path to the Ansible private data directory
        nm_path (str):          The path to the Net-Manage repository
        interface (str):        The interface (defaults to all interfaces)

    Returns:
        df_cam (DataFrame):     The CAM table and vendor OUI
    '''
    if interface:
        cmd = f'show mac address-table interface {interface}'
    else:
        cmd = 'show mac address-table'
    extravars = {'username': username,
                 'password': password,
                 'host_group': host_group,
                 'commands': cmd}
 
    # Execute the pre-checks
    playbook = f'{play_path}/cisco_ios_run_commands.yml'
    runner = ansible_runner.run(private_data_dir=private_data_dir,
                                playbook=playbook,
                                extravars=extravars)
    
    # Parse the output and add it to 'data'
    df_data = list()
    for event in runner.events:
        if event['event'] == 'runner_on_ok':
            event_data = event['event_data']
            
            device = event_data['remote_addr']
            
            output = event_data['res']['stdout'][0].split('\n')
            output = list(filter(None, output))
            pattern = '[a-zA-Z0-9]{4}\.[a-zA-Z0-9]{4}\.[a-zA-Z0-9]{4}'
            for line in output:
                for item in line.split():
                    valid = re.match(pattern, item)
                    if valid and line.split()[-1] != 'CPU':
                        mac = line.split()[1]
                        interface = line.split()[-1]
                        vlan = line.split()[0]
                        try:
                            vendor = find_mac_vendor(mac)
                        except Exception:
                            vendor = 'unknown'
                        df_data.append([device, interface, mac, vlan, vendor])
    
    # Define the dataframe columns
    cols = ['device',
            'interface',
            'mac',
            'vlan',
            'vendor']
    
    df_cam = pd.DataFrame(data=df_data, columns=cols)
    
    # Get the ARP table, map the IPs to the CAM table, and add them to 'df_cam'
    # TODO: Iterate through VRFs on devices that do not support 'vrf all'
    cmd = 'show ip arp'
    extravars = {'username': username,
                 'password': password,
                 'host_group': host_group,
                 'commands': cmd}

    # Execute 'show interface description' and parse the results
    playbook = f'{play_path}/cisco_ios_run_commands.yml'
    runner = ansible_runner.run(private_data_dir=private_data_dir,
                                playbook=playbook,
                                extravars=extravars)
    
    # Create a dict to store the ARP table
    ip_dict = dict()
    
    # Create a list to store the IP address for each MAC address. If there is
    # not an entry in the ARP table then an empty string will be added
    addresses = list()
    
    # Parse the output
    for event in runner.events:
        if event['event'] == 'runner_on_ok':
            event_data = event['event_data']
            device = event_data['remote_addr']
            
            output = event_data['res']['stdout'][0].split('\n')
            output = list(filter(None, output))
            for line in output[1:]:
                mac = line.split()[3]
                ip = line.split()[1]
                ip_dict[mac] = ip
                
            for idx, row in df_cam.iterrows():
                mac = row['mac']
                if ip_dict.get(mac):
                    addresses.append(ip_dict.get(mac))
                else:
                    addresses.append(str())
    
    # Add the addresses list to 'df_cam' as a column
    df_cam['ip'] = addresses
    
    # Set the desired order of columns
    cols = ['device',
            'interface',
            # 'description',
            'mac',
            # 'ip',
            'vlan',
            'vendor']
    df_cam = df_cam[cols]
    
    # Sort by interface
    df_cam = df_cam.sort_values('interface')
    
    # Reset (re-order) the index
    df_cam.reset_index(drop=True, inplace=True)
    
    return df_cam

## interface_descriptions

In [ ]:
def get_ios_inf_descs(username,
                      password,
                      host_group,
                      play_path,
                      private_data_dir,
                      interface=None):
    '''
    Gets IOS interface descriptions.

    Args:
        username (str):         The username to login to devices
        password (str):         The password to login to devices
        host_group (str):       The inventory host group
        play_path (str):        The path to the playbooks directory
        private_data_dir (str): The path to the Ansible private data directory
        interface (str):        The interface (defaults to all interfaces)

    Returns:
        df_desc (DataFrame):    The interface descriptions
    '''
    # Get the interface descriptions and add them to df_cam
    cmd = 'show interface description'
    extravars = {'username': username,
                 'password': password,
                 'host_group': host_group,
                 'commands': cmd}

    # Execute 'show interface description' and parse the results
    playbook = f'{play_path}/cisco_ios_run_commands.yml'
    runner = ansible_runner.run(private_data_dir=private_data_dir,
                                playbook=playbook,
                                extravars=extravars)
    # Create a list to store the rows for the dataframe
    df_data = list()
    for event in runner.events:
        if event['event'] == 'runner_on_ok':
            event_data = event['event_data']
            
            device = event_data['remote_addr']
            
            output = event_data['res']['stdout'][0].split('\n')
            # Get the position of the 'Description' column (we cannot split by
            # spaces because some interface descriptions have spaces in them).
            pos = output[0].index('Description')
            for line in output[1:]:
                inf = line.split()[0]
                desc = line[pos:]
                df_data.append([device, inf, desc])

    # Create the dataframe and return it
    cols = ['device', 'interface', 'description']
    df_desc = pd.DataFrame(data=df_data, columns=cols)
    return df_desc


## interface_IPs

In [ ]:
def cisco_ios_get_interface_ips(username,
                                password,
                                host_group,
                                play_path,
                                private_data_dir):
    '''
    Gets the IP addresses assigned to interfaces.

    Args:
        username (str):         The username to login to devices
        password (str):         The password to login to devices
        host_group (str):       The inventory host group
        play_path (str):        The path to the playbooks directory
        private_data_dir (str): The path to the Ansible private data directory

    Returns:
        df_ip (df):             A DataFrame containing the interfaces and IPs
    '''
    cmd = 'show ip interface | include line protocol|Internet address'
    extravars = {'username': username,
                 'password': password,
                 'host_group': host_group,
                 'commands': cmd}

    # Execute the command
    playbook = f'{play_path}/cisco_ios_run_commands.yml'
    runner = ansible_runner.run(private_data_dir=private_data_dir,
                                playbook=playbook,
                                extravars=extravars)

    # Parse the results
    ip_data = list()
    for event in runner.events:
        if event['event'] == 'runner_on_ok':
            event_data = event['event_data']
            
            device = event_data['remote_addr']
            
            output = event_data['res']['stdout'][0].split('\n')
            output.reverse()  # Reverse the output to make it easier to iterate
            for line in output:
                if 'Internet address' in line:
                    ip = line.split()[-1]
                    pos = output.index(line)+1
                    inf = output[pos].split()[0]
                    status = output[pos].split()[-1]
                    row = [device, inf, ip, status]
                    ip_data.append(row)
    # Create a dataframe from ip_data and return it
    cols = ['Device', 'Interface', 'IP', 'Status']
    df_ip = pd.DataFrame(data=ip_data, columns=cols)
    return df_ip

## find_uplink_by_ip

In [ ]:
def cisco_ios_find_uplink_by_ip(username,
                                password,
                                host_group,
                                play_path,
                                private_data_dir,
                                subnets=list()):
    '''
    Searches the hostgroup for a list of subnets (use /32 to esarch for a
    single IP). Once it finds them, it uses CDP and LLDP (if applicable) to try
    to find the uplink.
    
    If a list of IP addresses is not provided, it will attempt to find the
    uplinks for all IP addresses on the devices.
    
    This is a simple function that was writting for a single use case. It has
    some limitations:
    
    1. There is not an option to specify the VRF (although it will still return
       the uplinks for every IP that meets the parameters)
    2. If CDP and LLDP are disabled or the table is not populated, it does not
       try alternative methods like interface descriptions and CAM tables. I
       can add those if there is enough interest in this function.
    
    TODOs:
    - Add alternative methods if CDP and LLDP do not work:
      - Interface descriptions
      - Reverse DNS (in the case of P2P IPs)
      - CAM table
    - Add option to specify the VRF (low priority)

    Args:
        username (str):         The username to login to devices
        password (str):         The password to login to devices
        host_group (str):       The inventory host group
        play_path (str):        The path to the playbooks directory
        private_data_dir (str): The path to the Ansible private data directory
        addresses (list):       (Optional) A list of one or more subnets to
                                search for. Use CIDR notation. Use /32 to
                                search for individual IPs. If no list is
                                provided then the function will try to find the
                                uplinks for all IP addresses on the devices.

    Returns:
        df_combined (DF):       A DataFrame containing IP > remote port mapping
    '''
    
    # Get the IP addresses on the devices in the host group
    df_ip = cisco_ios_get_interface_ips(username,
                                        password,
                                        host_group,
                                        play_path,
                                        private_data_dir)

    # Get the CDP neighbors for the device
    df_cdp = get_ios_cdp_neighbors(username,
                                   password,
                                   host_group,
                                   play_path,
                                   private_data_dir)

    # Remove the sub-interfaces from df_ip
    local_infs = df_ip['Interface'].to_list()
    local_infs = [inf.split('.')[0] for inf in local_infs]
    df_ip['Interface'] = local_infs
    
    # Attempt to find the neighbors for the interfaces that have IPs
    df_data = list()

    for idx, row in df_ip.iterrows():
        device = row['Device']
        inf = row['Interface']
        neighbor_row = df_cdp.loc[(df_cdp['Device'] == device) &
                                  (df_cdp['Local Inf'] == inf)]
        remote_device = list(neighbor_row['Neighbor'].values)
        if remote_device:
            remote_device = remote_device[0]
            remote_inf = list(neighbor_row['Remote Inf'].values)[0]
        else:
            remote_device = 'unknown'
            remote_inf = 'unknown'
        mgmt_ip = row['IP']
        df_data.append([device, mgmt_ip, inf, remote_device, remote_inf])
    # Create a DataFrame and return it
    cols = ['Device',
            'IP',
            'Local Interface',
            'Remote Device',
            'Remote Interface']
    df_combined = pd.DataFrame(data=df_data, columns=cols)
    
    return df_combined

# NXOS Tests

## NXOS Pre- and Post-checks

Runs all standard NXOS Pre- and Post-checks

In [ ]:
def cisco_nxos_run_checks(username,
                          password,
                          host_group,
                          play_path,
                          private_data_dir):
    '''
    Run the pre- and post-checks on a Cisco NXOS and adds the results to
    a dictionary.
    
    Args:
        username (str):         The username to login to devices
        password (str):         The password to login to devices
        host_group (str):       The inventory host group
        play_path (str):        The path to the playbooks directory
        private_data_dir (str): The path to the Ansible private data directory

    Returns:
        results (dict):         A dictionary containing dataframes for each
                                check
    '''
    
    
    def nxos_checks_dump(check_data):
        '''
        Formats the output of the 'cisco_nxos_pre_post_checks.yml' playbook and
        returns it as a dictionary.

        Args:
            check_data (dict): The output of the playbooks

        Returns:
            formatted_checks (dict): A nested dictionary, where the value for each
                                     key is the formatted output
        '''

        nxos_devices = list()

        check_hostnames = dict()
        check_hostnames['inventory_name'] = list()
        check_hostnames['hostname'] = list()

        check_inf_status = dict()
        check_inf_status['hostname'] = list()
        check_inf_status['Port'] = list()
        check_inf_status['Name'] = list()
        check_inf_status['Status'] = list()
        check_inf_status['Vlan'] = list()
        check_inf_status['Duplex'] = list()
        check_inf_status['Speed'] = list()
        check_inf_status['Type'] = list()

        check_vpc_status = dict()
        check_vpc_status['hostname'] = list()
        check_vpc_status['id'] = list()
        check_vpc_status['Port'] = list()
        check_vpc_status['Status'] = list()
        check_vpc_status['Consistency'] = list()
        check_vpc_status['Reason'] = list()
        check_vpc_status['Active vlans'] = list()

        check_vpc_state = dict()
        check_vpc_state['hostname'] = list()
        check_vpc_state['vPC domain id'] = list()
        check_vpc_state['Peer status'] = list()
        check_vpc_state['vPC keep-alive status'] = list()
        check_vpc_state['Configuration consistency status'] = list()
        check_vpc_state['Per-vlan consistency status'] = list()
        check_vpc_state['Type-2 consistency status'] = list()
        check_vpc_state['vPC role'] = list()
        check_vpc_state['Number of vPCs configured'] = list()
        check_vpc_state['Peer Gateway'] = list()
        check_vpc_state['Peer gateway excluded VLANs'] = list()
        check_vpc_state['Dual-active excluded VLANs'] = list()
        check_vpc_state['Graceful Consistency Check'] = list()
        check_vpc_state['Operational Layer3 Peer-router'] = list()
        check_vpc_state['Auto-recovery status'] = list()

        # Extract the ARP table
        check_arp_table = dict()
        check_arp_table['hostname'] = list()
        check_arp_table['Address'] = list()
        check_arp_table['Age'] = list()
        check_arp_table['MAC Address'] = list()
        check_arp_table['Interface'] = list()

        # Extract the CAM table
        check_cam_table = dict()
        check_cam_table['hostname'] = list()
        check_cam_table['Legend'] = list()
        check_cam_table['VLAN'] = list()
        check_cam_table['MAC Address'] = list()
        check_cam_table['Type'] = list()
        check_cam_table['age'] = list()
        check_cam_table['Secure'] = list()
        check_cam_table['NTFY'] = list()
        check_cam_table['Ports/SWID.SSID.LID'] = list()

        # Extract the spanning-tree blocked ports
        check_stp_blocked = dict()
        check_stp_blocked['hostname'] = list()
        check_stp_blocked['Name'] = list()
        check_stp_blocked['Blocked Interfaces List'] = list()

        # Extract the total number of blocked STP ports
        check_total_stp_blocked = dict()
        check_total_stp_blocked['hostname'] = list()
        check_total_stp_blocked['Total Blocked Ports'] = list()

        # Extract the total number of err-disabled ports
        check_err_disabled = dict()
        check_err_disabled['hostname'] = list()
        check_err_disabled['Port'] = list()
        check_err_disabled['Name'] = list()
        check_err_disabled['Status'] = list()
        check_err_disabled['Reason'] = list()

        for key, value in check_data.items():
            # Extract the hostnames
            nxos_devices.append(key)
            check_hostnames['inventory_name'].append(key)
            check_hostnames['hostname'].append(value['show hostname'][0])

            header = value['show interface status err-disabled'][1]
            pos_port = header.index('Port')
            pos_name = header.index('Name')
            pos_status = header.index('Status')
            pos_reason = header.index('Reason')
            for line in value['show interface status err-disabled'][3:]:
                try:
                    port = line[pos_port:pos_name].strip()
                    name = line[pos_name:pos_status].strip()
                    status = line[pos_status:pos_reason].strip()
                    reason = line[pos_reason:].strip()
                    check_err_disabled['Port'].append(port)
                    check_err_disabled['Name'].append(name)
                    check_err_disabled['Status'].append(status)
                    check_err_disabled['Reason'].append(reason)
                    check_err_disabled['hostname'].append(key)
                except Exception:
                    pass

            num_err_disabled_ports = len(check_err_disabled.get('Port'))

            for line in value['show spanning-tree blockedports'][2:]:
                line = line.split()
                if len(line) == 2:
                    check_stp_blocked['hostname'].append(key)
                    check_stp_blocked['Name'].append(line[0])
                    check_stp_blocked['Blocked Interfaces List'].append(line[1])
                if '(segments)' in line:
                    check_total_stp_blocked['hostname'].append(key)
                    check_total_stp_blocked['Total Blocked Ports'].append(line[-1])

            for line in value['show ip arp vrf all | begin "MAC Address"'][1:]:
                check_arp_table['hostname'].append(key)
                check_arp_table['Address'].append(line.split()[0])
                check_arp_table['Age'].append(line.split()[1])
                check_arp_table['MAC Address'].append(line.split()[2])
                check_arp_table['Interface'].append(line.split()[3])        

            for line in value['show mac address-table  | begin "MAC Address"'][2:]:
                line = line.split()
                if len(line) == 8:
                    check_cam_table['Legend'].append(line[0])
                    check_cam_table['VLAN'].append(line[1])
                    check_cam_table['MAC Address'].append(line[2])
                    check_cam_table['Type'].append(line[3])
                    check_cam_table['age'].append(line[4])
                    check_cam_table['Secure'].append(line[5])
                    check_cam_table['NTFY'].append(line[6])
                    check_cam_table['Ports/SWID.SSID.LID'].append(line[7])
                    check_cam_table['hostname'].append(key)

            # Extract the interface statuses
            header = value['show interface status'][1]
            pos = header.index('Status')
            for line in value['show interface status'][3:]:
                # Get the interface
                port = line.split()[0]

                # Get the interface name (this accounts for devices that change the
                # starting position of columns)
                line_prefix = line[:pos].split()
                name = ' '.join(line_prefix[1:])

                # Get the remaining parameters
                line_suffix = line[pos:].split()
                status = line_suffix[0]
                vlan = line_suffix[1]
                duplex = line_suffix[2]
                speed = line_suffix[3]
                try:
                    port_type = line_suffix[4]
                except Exception:
                    port_type = str()

                check_inf_status['hostname'].append(key)
                check_inf_status['Port'].append(port)
                check_inf_status['Name'].append(name)
                check_inf_status['Status'].append(status)
                check_inf_status['Vlan'].append(vlan)
                check_inf_status['Duplex'].append(duplex)
                check_inf_status['Speed'].append(speed)
                check_inf_status['Type'].append(port_type)

            # Extract the VPC state information
            vpc_state = value['show vpc brief | begin "vPC domain id" | end "vPC Peer-link status"'][:-2]
            check_vpc_state['hostname'].append(key)
            for line in vpc_state:
                line = line.split(':')
                line = [l.strip() for l in line]
                check_vpc_state[line[0]].append(line[-1])
            # Remove empty keys in vpc_state (e.g., if 'peer gateway' is not
            # configured then there will not be any Peer gateway excluded VLANs
            to_remove = list()
            for _key, _value in check_vpc_state.items():
                if not _value:
                    to_remove.append(_key)
            for item in to_remove:
                del check_vpc_state[item]
            
            vpc_status = value['show vpc brief | begin "vPC status"'][4:]
            vpc_status = [l for l in vpc_status if len(l.split()) != 1]
            for line in vpc_status:
                line = line.split()
                if len(line) == 6 or len(line) == 8:
                    id_ = line[0]
                    port = line[1]
                    status = line[2]
                    if len(line) == 8:
                        consistency = 'Not Applicable'
                        reason = 'Consistency Check Not Performed'
                    else:
                        consistency = line[3]
                        reason = line[4]
                    active_vlans = line[-1]
                    check_vpc_status['hostname'].append(key)
                    check_vpc_status['id'].append(id_)
                    check_vpc_status['Port'].append(port)
                    check_vpc_status['Status'].append(status)
                    check_vpc_status['Consistency'].append(consistency)
                    check_vpc_status['Reason'].append(reason)
                    check_vpc_status['Active vlans'].append(active_vlans)        

        if check_vpc_state.get('Peer Gateway'):
            vpc_in_use = True
        else:
            vpc_in_use = False

        formatted_checks = dict()
        formatted_checks['check_arp_table'] = check_arp_table
        formatted_checks['check_cam_table'] = check_cam_table
        formatted_checks['check_err_disabled'] = check_err_disabled
        formatted_checks['num_err_disabled_ports'] = num_err_disabled_ports
        formatted_checks['check_hostnames'] = check_hostnames
        formatted_checks['check_inf_status'] = check_inf_status
        formatted_checks['check_stp_blocked'] = check_stp_blocked
        formatted_checks['check_total_stp_blocked'] = check_total_stp_blocked
        formatted_checks['check_vpc_status'] = check_vpc_status
        formatted_checks['check_vpc_state'] = check_vpc_state
        formatted_checks['vpc_in_use'] = vpc_in_use

        return formatted_checks    
    
    # Create the base variables
    check_data = dict()
    
    # Define the commands to run
    commands = {'hostname_command': 'show hostname',
                'diff_command': 'show running-config diff',
                'interface_status_command': 'show interface status',
                'err_disabled_command': 'show interface status err-disabled',
                'logging_command': 'show logging last 9999 | grep "err-disable\|BPDU"',
                'stp_blocked_ports_command': 'show spanning-tree blockedports',
                'vpc_state_command': 'show vpc brief | begin "vPC domain id" | end "vPC Peer-link status"',
                'vpc_status_command': 'show vpc brief | begin "vPC status"',
                'vpc_consistency_command': 'show vpc consistency-parameters global | grep "Local suspended"',
                'vpc_keepalive_command': 'show vpc peer-keepalive | grep -v "ms\|msec"',
                'get_arp_table_command': 'show ip arp vrf all | begin "MAC Address"',
                'get_cam_table_command': 'show mac address-table  | begin "MAC Address"'}

    # Create the extra variables to pass to Ansible
    extravars = {'username': username,
                 'password': password,
                 'host_group': host_group}

    for key, value in commands.items():
        extravars[key] = value
    
    # Execute the pre-checks
    runner = ansible_runner.run(private_data_dir=private_data_dir,
                                playbook=f'{play_path}/cisco_nxos_pre_post_checks.yml',
                                extravars=extravars)
    
    # Parse the output; add the hosts to 'devices' and the command output to 'check_data'
    for event in runner.events:
        if event['event'] == 'runner_on_ok':
            event_data = event['event_data']
            device = event_data['remote_addr']
            
            # Create the dictionary keys for 'check_data'
            if not check_data.get(device):
                check_data[device] = dict()
            
            # Add the command output to 'check_data'
            cmd = event_data['res']['invocation']['module_args']['commands'][0]
            output = event_data['res']['stdout'][0].split('\n')
            check_data[device][cmd] = output

    # Format the output of check_data
    # TODO: The above code needs to be refactored. It is a mess right now.
    nxos_checks_dump(check_data)

    # Create dataframes for the output of each of the checks
    pre_formatted_checks = nxos_checks_dump(check_data)
    pre_check_arp_table = pre_formatted_checks['check_arp_table']
    pre_check_cam_table = pre_formatted_checks['check_cam_table']
    pre_check_err_disabled = pre_formatted_checks['check_err_disabled']
    pre_check_num_err_disabled_ports = pre_formatted_checks['num_err_disabled_ports']
    pre_check_hostnames = pre_formatted_checks['check_hostnames']
    pre_check_inf_status = pre_formatted_checks['check_inf_status']
    pre_check_stp_blocked = pre_formatted_checks['check_stp_blocked']
    pre_check_total_stp_blocked = pre_formatted_checks['check_total_stp_blocked']
    pre_check_vpc_status = pre_formatted_checks['check_vpc_status']
    pre_check_vpc_state = pre_formatted_checks['check_vpc_state']
    vpc_in_use = pre_formatted_checks['vpc_in_use']
    
    df_pre_arp_table = pd.DataFrame.from_dict(pre_check_arp_table)
    del df_pre_arp_table['Age']

    df_pre_hostnames = pd.DataFrame.from_dict(pre_check_hostnames)
    # checks['nxos']['hostnames'] = df_pre_hostnames
    # display(df_pre_hostnames)

    if pre_check_num_err_disabled_ports:
        df_pre_err_disabled = pd.DataFrame.from_dict(pre_check_err_disabled)
        # checks['nxos']['err_disabled_ports'] = df_pre_err_disabled
        display(df_pre_err_disabled)

    df_pre_check_stp_blocked = pd.DataFrame.from_dict(pre_check_stp_blocked)
    # checks['nxos']['stp_blocked_ports'] = df_pre_check_stp_blocked
    # display(df_pre_check_stp_blocked)

    df_pre_check_total_stp_blocked = pd.DataFrame.from_dict(pre_check_total_stp_blocked)
    # checks['nxos']['total_stp_blocked_ports'] = df_pre_check_total_stp_blocked
    # display(df_pre_check_total_stp_blocked)

    df_pre_inf_status = pd.DataFrame.from_dict(pre_check_inf_status)
    # checks['nxos']['interface_statuses'] = df_pre_inf_status
    # display(df_pre_inf_status)

    df_arp_table = pd.DataFrame.from_dict(pre_check_arp_table)
    # checks['nxos']['arp_table'] = df_arp_table

    df_pre_cam_table = pd.DataFrame.from_dict(pre_check_cam_table)
    # checks['nxos']['cam_table'] = df_pre_cam_table
    # display(df_pre_cam_table)

    if vpc_in_use:
        df_pre_vpc_state = pd.DataFrame.from_dict(pre_check_vpc_state)
        # checks['nxos']['vpc_state'] = df_pre_vpc_state
        # display(df_pre_vpc_state)

        df_pre_vpc_status = pd.DataFrame.from_dict(pre_check_vpc_status)
        # checks['nxos']['vpc_status'] = df_pre_vpc_status
        # display(df_pre_vpc_status)

    # Create a dictionary of dataframes to return
    nxos_checks = dict()
    nxos_checks['df_arp_table'] = df_pre_arp_table
    nxos_checks['df_hostnames'] = df_pre_hostnames
    if pre_check_num_err_disabled_ports:
        nxos_checks['df_err_disabled'] = df_pre_err_disabled
    nxos_checks['df_check_stp_blocked'] = df_pre_check_stp_blocked
    nxos_checks['df_check_total_stp_blocked'] = df_pre_check_total_stp_blocked
    nxos_checks['df_inf_status'] = df_pre_inf_status
    nxos_checks['df_arp_table'] = df_arp_table
    nxos_checks['df_cam_table'] = df_pre_cam_table
    if vpc_in_use:
        nxos_checks['df_vpc_state'] = df_pre_vpc_state
        nxos_checks['df_vpc_status'] = df_pre_vpc_status

    return nxos_checks

## cam_table

In [1]:
def get_nxos_cam_table(username,
                       password,
                       host_group,
                       play_path,
                       private_data_dir,
                       nm_path,
                       interface=None):
    '''
    Gets the CAM table for NXOS devicecs and adds the vendor OUI.

    Args:
        username (str):         The username to login to devices
        password (str):         The password to login to devices
        host_group (str):       The inventory host group
        play_path (str):        The path to the playbooks directory
        private_data_dir (str): The path to the Ansible private data directory
        interface (str):        The interface (defaults to all interfaces)
        nm_path (str):          The path to the Net-Manage repository

    Returns:
        df_cam (DataFrame):     The CAM table and vendor OUI
    '''
    # Create a list of vendor OUIs
    df_ouis = update_ouis(nm_path)
    
    if interface:
        cmd = f'show mac address-table interface {interface}'
    else:
        cmd = 'show mac address-table'
    extravars = {'username': username,
                 'password': password,
                 'host_group': host_group,
                 'commands': cmd}
 
    # Execute the pre-checks
    playbook = f'{play_path}/cisco_nxos_run_commands.yml'
    runner = ansible_runner.run(private_data_dir=private_data_dir,
                                playbook=playbook,
                                extravars=extravars)

    # Define the RegEx pattern for a valid MAC address
    # pattern = '([0-9a-f]{4}\.[0-9a-f]{4}\.[0-9a-f]{4})'
    pattern = '.*[a-zA-Z0-9]{4}\.[a-zA-Z0-9]{4}\.[a-zA-Z0-9]{4}.*'
    
    # Create a list to store MAC addresses
    addresses = list()
    
    # Parse the output and add it to 'data'
    df_data = list()
    
    for event in runner.events:
        if event['event'] == 'runner_on_ok':
            event_data = event['event_data']
            
            device = event_data['remote_addr']
            
            # output = event_data['res']['stdout'][0].split('\n')
            output = event_data['res']['stdout'][0]
            
            output = re.findall(pattern, output)
            for line in output:
                mac = line.split()[2]
                interface = line.split()[-1]
                vlan = line.split()[1]
                # if not uniques.get(mac):
                #     try:
                #         vendor = find_mac_vendor(mac)
                #     except Exception:
                #         vendor = 'unknown'
                #     uniques[mac] = vendor
                # else:
                #     vendor = uniques.get(mac)
                
                # print(line)
                df_data.append([device, interface, mac, vlan])
                
                    
                # data['device'].append(device)
                # data['mac'].append(mac)
                # data['interface'].append(interface)
                # data['vendor'].append(vendor)
                # data['vlan'].append(vlan)
    # Create the dataframe and return it
    cols = ['device',
            'interface',
            'mac',
            'vlan']    
    df_cam = pd.DataFrame(data=df_data, columns=cols)

    # Get the OUIs and add them to df_cam
    addresses = df_cam['mac'].to_list()
    vendors = find_mac_vendor(addresses, df_ouis)
    df_cam['vendor'] = vendors
    
    # Return df_cam
    return df_cam